In [1]:
import pandas as pd
import numpy as np


In [3]:
path = "/content/drive/MyDrive/Dataset For Practice/NaiveBayes.csv"
dataset = pd.read_csv(path, delimiter=',')
dataset

,day,season,wind,rain,class
0,weekday,spring,none,none,ontime
1,weekday,winter,none,slight,ontime
2,weekday,winter,none,slight,ontime
3,weekday,winter,high,heavy,late
4,saturday,summer,normal,none,ontime
5,weekday,autumn,normal,none,verylate
6,holiday,summer,high,slight,ontime
7,sunday,summer,normal,none,ontime
8,weekday,winter,high,heavy,verylate
9,weekday,summer,none,slight,ontime


In [8]:
dataset.columns = dataset.columns.str.replace(' ', '')

In [6]:
classes = list(dataset["class"].unique())
classes

['ontime', 'late', 'verylate', 'cancelled']

In [9]:
attributes = []
attributes = list(dataset.columns[:-1])
for a in attributes:
  dataset[a] = dataset[a].str.strip() #removing spaces from field

In [73]:
class NaiveBayesClassifier:
  def __init__(self, dataset, classes, attributes):
    self.dataset = dataset
    self.classes = classes
    self.attributes = attributes
    self.length = len(dataset)
    self.attribiuteProb = {}
    self.classProbability = {}
    self.classCount = {}
    self.attributeDict = {}
    self.clsPosteriroProb = {i:0 for i in classes}

    for a in attributes:
      self.attributeDict[a] = list(dataset[a].unique())
    for c in classes:
      self.classCount[c] = dataset["class"].value_counts()[c]

    
  def priorProbability(self):
    for c in classes:
      self.classProbability[c] = dataset["class"].value_counts()[c]/self.length
    return self.classProbability
  
  def conditionalProbability(self):
    for key, values in self.attributeDict.items():
      for value in values:
        att = []
        for c in classes:
          attributeFreq = len(dataset[(dataset[key] == value) & (dataset["class"]==c)]) #attribute frequency for per class
          computedProbability = attributeFreq/self.classCount[c]
          att.append(computedProbability)
          self.attribiuteProb[value] = att
    self.posteriorDF = pd.DataFrame.from_dict(self.attribiuteProb, orient='index', columns = classes)
    return self.posteriorDF

  def posteriorProbability(self,attributeInInteres):
    for c in classes:
      probabilityProduct = 1
      self.posteriorDF = self.conditionalProbability()
      self.classProbability = self.priorProbability()
      
      for attribute in attributeInInteres:
          probabilityProduct *= self.posteriorDF.loc[attribute][c] #loc[name][index]
          self.clsPosteriroProb[c] = probabilityProduct * self.classProbability[c]

    return self.clsPosteriroProb
  
  def predictedClass(self,attributeInInteres):
      self.clsPosteriroProb = self.posteriorProbability(attributeInInteres)
      predictedMaxValue = max(self.clsPosteriroProb.values())
      predictedMaxValue
      #EXTRACTING key FROM DICTIONARY USING value
      predictedClass = list(self.clsPosteriroProb.keys())[list(self.clsPosteriroProb.values()).index(predictedMaxValue)]

      return predictedClass

  def normalization(self, attributeInInteres):
      self.clsPosteriroProb = self.posteriorProbability(attributeInInteres)
      posteriorValues = self.clsPosteriroProb.values()
      normalizedPosValue = [value/sum(posteriorValues) for value in posteriorValues]
      maxNormalizedVal = max(normalizedPosValue)
      predictedNormClass = list(self.clsPosteriroProb.keys())[normalizedPosValue.index(maxNormalizedVal)]
      return predictedNormClass
       





In [77]:
nb = NaiveBayesClassifier(dataset, classes, attributes)
attributeInInteres = ['saturday', 'spring', 'high', 'heavy']
nb.predictedClass(attributeInInteres)


'cancelled'

In [76]:
nb.normalization(attributeInInteres)

'cancelled'